In [2]:
#!pip install imagehash

  Using cached ImageHash-4.3.1-py2.py3-none-any.whl.metadata (8.0 kB)
Using cached ImageHash-4.3.1-py2.py3-none-any.whl (296 kB)


In [4]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import json
import os
import matplotlib.pylab as plt
import csv
from scipy.cluster.hierarchy import linkage, fcluster, dendrogram
from sklearn.cluster import DBSCAN
import shutil
from PIL import Image
import imagehash
import cv2
from matplotlib.pyplot import imread
import random

from sklearn.metrics import silhouette_score, silhouette_samples


In [5]:
#train_path = "C:\\Users\\USER\\Documents\\VM-shared_177\\extracted_APIDMS\\APIDMS_img2\\train"
#train_dest = "C:\\Users\\USER\\Documents\\VM-shared_177\\extracted_APIDMS\\APIDMS_img2\\train3"

#train_path = "C:\\Users\\USER\\Documents\\VM-shared_177\\vs0177_full_split1\\train"
#train_dest = "C:\\Users\\USER\\Documents\\VM-shared_177\\vs0177_full_split1\\train3"

# train_path = "train"
# train_dest = "train_sub"

#train_dest = "C:\\Users\\USER\\Documents\\VM-shared_177\\report_117\\extracted_report\\report177_img2\\train4"

# val_path = "C:\\Users\\USER\\Documents\\VM-shared_177\\report_117\\split3\\validate"
# val_dest = "C:\\Users\\USER\\Documents\\VM-shared_177\\report_117\\split3\\validate1"

# test_path = "C:\\Users\\USER\\Documents\\VM-shared_177\\report_117\\split3\\validate1"

# Define the weights for each distance matrix

weight_avg = 0.5 
weight_color = 0.5  

# Define the threshold for clustering
threshold_cluster = 20
threshold_combine = 20

In [6]:
import numpy as np
import os
from PIL import Image
import imagehash
from sklearn.cluster import DBSCAN

## Clustering with DBSCAN
def get_hashes(img):
    phash = imagehash.phash(img, hash_size=8)
    chash = imagehash.colorhash(img)
    return np.array(phash.hash.flatten()), np.array(chash.hash.flatten())  # Flatten and return as np.array for easier manipulation

def clustering_family(fam_path, dest, eps=2, min_samples=1):
    images = []
    img_hashes = []
    img_paths = []

    for img in os.listdir(fam_path):
        image_path = os.path.join(fam_path, img)
        image = Image.open(image_path)
        phash, chash = get_hashes(image)
        combined_hash = np.concatenate((phash, chash))  # Combine the two hashes
        images.append(image)
        img_hashes.append(combined_hash)
        img_paths.append(image_path)

    img_hashes = np.array(img_hashes)  # Convert list of hashes to a numpy array for DBSCAN

    if len(images) > 1:
        db = DBSCAN(eps=eps, min_samples=min_samples, metric="euclidean").fit(img_hashes)
        labels = db.labels_

        # Create clusters
        sub_families = {}
        for img_path, cluster in zip(img_paths, labels):
            if cluster == -1:
                continue  # Skip noise points
            if cluster not in sub_families:
                sub_families[cluster] = []
            sub_families[cluster].append(img_path)
    else:
        sub_families = {1: img_paths}

    # Create sub-family folders and move/copy images
    for cluster, images in sub_families.items():
        fam_name = os.path.basename(fam_path)
        sub_family_folder = os.path.join(dest, fam_name)
        sub_family_folder = os.path.join(sub_family_folder, fam_name + str(cluster))
        if not os.path.exists(sub_family_folder):
            os.makedirs(sub_family_folder)

        for image_path in images:
            image_filename = os.path.basename(image_path)
            new_image_path = os.path.join(sub_family_folder, image_filename)
            if fam_path == dest:  # Move the file
                shutil.move(image_path, new_image_path)
            else:  # Copy the file
                shutil.copy(image_path, new_image_path)

In [7]:
#====================================================================================
#                   Begin Clustering process (Training data) - into sub-folders
#====================================================================================
# Remove subdirectory and its contents in the train_dest path
if os.path.isdir(train_dest):
    shutil.rmtree(train_dest)

# Prceed all families (folders)
print('[Info] Families Clustering...')
for fam in os.listdir(train_path):
    fam_path = os.path.join(train_path,fam)
    #print('family:',fam)
    clustering_family(fam_path,train_dest)

[Info] Families Clustering...


Image Augmentation

In [19]:
import os
import random
import numpy as np
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm

def perform_augmentation(extract, src, generator):
    total_image = 20
    print('[Info] Sub-Families Augmentation Begin...')
    for dir_fam in tqdm(os.listdir(src), desc='Family level'):
        sub_dir = os.path.join(src, dir_fam)
        extract_fam = os.path.join(extract, dir_fam)

        for sub_fam in (os.listdir(sub_dir)):
            sub_fam_path = os.path.join(sub_dir, sub_fam)
            file_list = os.listdir(sub_fam_path)
            image_need = total_image - len(file_list) if len(file_list) < total_image else 0

            # Perform Augmentation
            for num in tqdm(range(image_need, 0, -1), desc= f"Augmenting on - {sub_fam}"):
                selected_img = random.choice(file_list)
                md5 = '.'.join(selected_img.split('.')[:3])  # Extract the prefix part before the first dot

                # Read the based file for further augmentation
                base_extract = os.path.join(extract_fam, f"{md5}.png")
                
                image = cv2.imread(base_extract)

                # Decide whether to do jittering or not
                jitter = random.choice([True, False])
                aug_name = f"{md5}_{num}"

                generator.generate_img_from_array(image, sub_fam_path, aug_name, jitter)

class ImageGenerator:
    def __init__(self, width, height):
        self.width = width
        self.height = height

    def generate_img_from_array(self, image_array, output_path, file_name, jitter):
        if jitter:
            image_array = self.apply_jitter(image_array)

        # Save the image
        output_file = f"{output_path}/{file_name}.png"
        plt.imsave(output_file, image_array)
        print(f"Saved augmented image: {output_file}")

    def apply_jitter(self, image):
        # Apply jittering (e.g., random color changes)
        jitter_value = 20
        jitter_matrix = np.random.randint(-jitter_value, jitter_value, image.shape, dtype=np.int16)
        jittered_image = np.clip(image.astype(np.int16) + jitter_matrix, 0, 255).astype(np.uint8)
        return jittered_image

# Define the path to the extracted training data
extract_train_folder = "train"
train_sub_path = "train_sub"

def get_ImageGenerator(width, height):
    return ImageGenerator(width, height)

# Initialize the generator
generator = get_ImageGenerator(256, 256)

# Assuming extract_train_folder and train_sub_path are defined
perform_augmentation(extract_train_folder, train_sub_path, generator)

[Info] Sub-Families Augmentation Begin...


Family level:   0%|          | 0/24 [00:00<?, ?it/s]

Saved augmented image: train_sub\AGENTTESLA\AGENTTESLA0/240610-nbmtnahb86.behavioral1.pcapng_13.png
Saved augmented image: train_sub\AGENTTESLA\AGENTTESLA0/240610-m32w8agd2w.behavioral1.pcapng_12.png
Saved augmented image: train_sub\AGENTTESLA\AGENTTESLA0/240610-m32w8agd2w.behavioral2.pcapng_11.png
Saved augmented image: train_sub\AGENTTESLA\AGENTTESLA0/240610-nb3vwagf4y.behavioral1.pcapng_10.png
Saved augmented image: train_sub\AGENTTESLA\AGENTTESLA0/240610-m5m6vagd8s.behavioral2.pcapng_9.png
Saved augmented image: train_sub\AGENTTESLA\AGENTTESLA0/240610-m5m6vagd8s.behavioral2.pcapng_8.png
Saved augmented image: train_sub\AGENTTESLA\AGENTTESLA0/240610-m32w8agd2w.behavioral2.pcapng_7.png


Augmenting on - AGENTTESLA0: 100%|██████████| 13/13 [00:00<00:00, 45.84it/s]


Saved augmented image: train_sub\AGENTTESLA\AGENTTESLA0/240610-m5m6vagd8s.behavioral2.pcapng_6.png
Saved augmented image: train_sub\AGENTTESLA\AGENTTESLA0/240610-nbmtnahb86.behavioral1.pcapng_5.png
Saved augmented image: train_sub\AGENTTESLA\AGENTTESLA0/240610-m5m6vagd8s.behavioral2.pcapng_4.png
Saved augmented image: train_sub\AGENTTESLA\AGENTTESLA0/240610-m32w8agd2w.behavioral2.pcapng_3.png
Saved augmented image: train_sub\AGENTTESLA\AGENTTESLA0/240610-m32w8agd2w.behavioral2.pcapng_2.png
Saved augmented image: train_sub\AGENTTESLA\AGENTTESLA0/240610-m5m6vagd8s.behavioral1.pcapng_1.png


...

Augmenting on - ZLOADER8: 100%|██████████| 14/14 [00:00<00:00, 37.31it/s]

Saved augmented image: train_sub\ZLOADER\ZLOADER8/240428-cb55fsdf37.behavioral2.pcapng_1.png


Saved augmented image: train_sub\ZLOADER\ZLOADER9/240409-pvg8ysfb33.behavioral1.pcapng_19.png
Saved augmented image: train_sub\ZLOADER\ZLOADER9/240409-pvg8ysfb33.behavioral1.pcapng_18.png
Saved augmented image: train_sub\ZLOADER\ZLOADER9/240409-pvg8ysfb33.behavioral1.pcapng_17.png
Saved augmented image: train_sub\ZLOADER\ZLOADER9/240409-pvg8ysfb33.behavioral1.pcapng_16.png
Saved augmented image: train_sub\ZLOADER\ZLOADER9/240409-pvg8ysfb33.behavioral1.pcapng_15.png
Saved augmented image: train_sub\ZLOADER\ZLOADER9/240409-pvg8ysfb33.behavioral1.pcapng_14.png
Saved augmented image: train_sub\ZLOADER\ZLOADER9/240409-pvg8ysfb33.behavioral1.pcapng_13.png
Saved augmented image: train_sub\ZLOADER\ZLOADER9/240409-pvg8ysfb33.behavioral1.pcapng_12.png
Saved augmented image: train_sub\ZLOADER\ZLOADER9/240409-pvg8ysfb33.behavioral1.pcapng_11.png
Saved augmented image: train_sub\ZLOADER\ZLOADER9/240409-pvg8ysfb33.behavioral1.pcapng_10.png
Saved augmented image: train_sub\ZLOADER\ZLOADER9/240409-pvg

Family level: 100%|██████████| 24/24 [02:28<00:00,  6.18s/it]

Saved augmented image: train_sub\ZLOADER\ZLOADER9/240409-pvg8ysfb33.behavioral1.pcapng_6.png
Saved augmented image: train_sub\ZLOADER\ZLOADER9/240409-pvg8ysfb33.behavioral1.pcapng_5.png
Saved augmented image: train_sub\ZLOADER\ZLOADER9/240409-pvg8ysfb33.behavioral1.pcapng_4.png
Saved augmented image: train_sub\ZLOADER\ZLOADER9/240409-pvg8ysfb33.behavioral1.pcapng_3.png
Saved augmented image: train_sub\ZLOADER\ZLOADER9/240409-pvg8ysfb33.behavioral1.pcapng_2.png
Saved augmented image: train_sub\ZLOADER\ZLOADER9/240409-pvg8ysfb33.behavioral1.pcapng_1.png
